In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append("/dfs/scratch0/lorr1/projects/bootleg/tutorials")
from utils import score_predictions, load_train_data
import pandas as pd
import time
import numpy as np
import cytoolz as tz
import feather
import ujson as json
from multiprocessing import  Pool
import jsonlines
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
from IPython.core.display import display, HTML, Markdown
from bootleg.symbols.entity_symbols import EntitySymbols
from bootleg.symbols.type_symbols import TypeSymbols
from bootleg.symbols.kg_symbols import KGSymbols
def printmd(string):
    display(Markdown(string))
tqdm.pandas()
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

In [3]:
def accuracy(df, crc_col="gold_qid", pred_col="pred_qid"):
    total = df.shape[0]
    correct = df[df[crc_col] == df[pred_col]].shape[0]
    return correct/total if total > 0 else 0

def get_incorrect(df, crc_col="gold_qid", pred_col="pred_qid"):
    return df[df[crc_col] != df[pred_col]]

def num_incorrect(df, crc_col="gold_qid", pred_col="pred_qid"):
    return df[df[crc_col] != df[pred_col]].shape[0]

def print_incorrect_over_total(df, crc_col="gold_qid", pred_col="pred_qid"):
    num_in = df[df[crc_col] != df[pred_col]].shape[0]
    total = df.shape[0]
    print(f"{num_in} / {total} = {num_in/total}")

def errors_by_type(df, type_sys, pred_col="pred_qid"):
    errors_type = defaultdict(int)
    df["correct"] = df["gold_qid"] == df[pred_col]
    for r in df.iterrows():
        row = r[1]
        if row.correct is True:
            continue
        for t in row[type_sys]:
            errors_type[t] += 1
    return errors_type

def apply_lfs(df, lfs):
    for lf in lfs:
        df[f"lf_{lf.__name__}"] = df.progress_apply(lf, axis=1)
        subset = df[df[f"lf_{lf.__name__}"]]
        lf_acc = accuracy(subset)
        print(f"LF: {lf.__name__} Acc: {lf_acc} Supp: {subset.shape[0]} Overall Acc: {accuracy(df)} Overall Size: {df.shape[0]}")
    return df

In [4]:
input_dir = Path('/dfs/scratch0/lorr1/projects/bootleg-data/data/korealiases_title_0122/')
a2q = json.load(open(input_dir / "entity_db/entity_mappings/alias2qids.json"))
entity_dump = EntitySymbols(load_dir=input_dir / "entity_db/entity_mappings")
emb_dir = Path('/dfs/scratch0/lorr1/projects/bootleg-data/embs')
types_hy = TypeSymbols(entity_dump, emb_dir, max_types=3, type_vocab_file="hyena_vocab.json", type_file="hyena_types_1229.json")
types_wd = TypeSymbols(entity_dump, emb_dir, max_types=3, type_vocab_file="wikidatatitle_to_typeid_1229.json", type_file="wikidata_types_1229.json")
types_rel = TypeSymbols(entity_dump, emb_dir, max_types=50, type_vocab_file="relation_to_typeid_1229.json", type_file="kg_relation_types_1229.json")
kg_syms = KGSymbols(entity_dump, emb_dir, "kg_adj_1229.txt")
q2title = json.load(open(input_dir / "entity_db/entity_mappings/qid2title.json"))
title2q = {v:k for k,v in q2title.items()}

Loading types from /dfs/scratch0/lorr1/projects/bootleg-data/embs/hyena_types_1229.json


Reading /dfs/scratch0/lorr1/projects/bootleg-data/embs/hyena_types_1229.json: 100%|██████████| 5832699/5832699 [00:11<00:00, 525424.32it/s]


Loading types from /dfs/scratch0/lorr1/projects/bootleg-data/embs/wikidata_types_1229.json


Reading /dfs/scratch0/lorr1/projects/bootleg-data/embs/wikidata_types_1229.json: 100%|██████████| 5832699/5832699 [00:10<00:00, 546280.42it/s]


Loading types from /dfs/scratch0/lorr1/projects/bootleg-data/embs/kg_relation_types_1229.json


Reading /dfs/scratch0/lorr1/projects/bootleg-data/embs/kg_relation_types_1229.json: 100%|██████████| 5832699/5832699 [00:11<00:00, 491232.69it/s]


Loading kg adj from /dfs/scratch0/lorr1/projects/bootleg-data/embs/kg_adj_1229.txt


100%|██████████| 29745502/29745502 [00:54<00:00, 542332.55it/s]


In [5]:
qid2cnt = defaultdict(int)
alias2qid2count = defaultdict(lambda: defaultdict(int))
with jsonlines.open(input_dir / "train.jsonl") as in_f:
    for line in in_f:
        for al, qid in zip(line["aliases"], line["qids"]):
            qid2cnt[qid] += 1
            alias2qid2count[al][qid] += 1
qid2cnt = dict(qid2cnt)
alias2qid2count = dict(alias2qid2count)
with open(input_dir / "train_qidcnt.json", "w") as out_f:
    json.save(qid2cnt, out_f)
with open(input_dir / "train_alias2qidcnt.json", "w") as out_f:
    json.save(alias2qid2count, out_f)

In [6]:
columns = ["sentence", "sent_idx", "is_gold_label", "alias", "span", "gold_title", "gold_qid", "pred_title", "in_cand", "pred_qid",
           "qid_cnt", "pred_qid_cnt", "aliases", "wikidata_types_1229_gld", "wikidata_types_1229_pred", "cand_names", "cand_probs"]

In [7]:
# Read in train data
# Load training data and split
if os.path.exists("saved_data/kore_train_df.feather"):
    train_df = feather.read_dataframe("saved_data/kore_train_df.feather")
else:
    train_df =  load_train_data(input_dir / "train.jsonl", q2title, a2q, type_symbols=[types_wd, types_hy], kg_symbols=[kg_syms])
    train_df["sentence_split"] = train_df["sentence"].apply(lambda x: x.split())
    feather.write_dataframe(train_df, "saved_data/kore_train_df.feather")

100%|██████████| 637185/637185 [03:28<00:00, 3050.71it/s] 


In [ ]:
mods = {}
pred_file = '/dfs/scratch1/lorr1/projects/bootleg/logs_guid/base/2021_01_23/23_45_31/2f2b98c2/test/last_model/bootleg_labels.jsonl'

boot_df = score_predictions(orig_file=input_dir / 'test.jsonl',
                 pred_file=pred_file,
                 title_map=q2title,
                 cands_map=a2q,
                 type_symbols=[types_hy, types_wd, types_rel],
                 kg_symbols=[kg_syms])
mods["boot"] = boot_df

pred_file2 = '/dfs/scratch1/lorr1/projects/bootleg/logs_guid/ent_only/2021_01_23/23_48_06/75fdf972/test/last_model/bootleg_labels.jsonl'

boot_df_ent = score_predictions(orig_file=input_dir / 'test.jsonl',
                 pred_file=pred_file2,
                 title_map=q2title,
                 cands_map=a2q,
                 type_symbols=[types_hy, types_wd, types_rel],
                 kg_symbols=[kg_syms])
mods["ent"] = boot_df_ent

pred_file = '/dfs/scratch1/lorr1/projects/bootleg/logs_guid/base_wl_socc_air/2021_01_30/16_39_08/ed7aec6f/test/last_model/bootleg_labels.jsonl'

guid_df = score_predictions(orig_file=input_dir / 'test.jsonl',
                 pred_file=pred_file,
                 title_map=q2title,
                 cands_map=a2q,
                 type_symbols=[types_hy, types_wd, types_rel],
                 kg_symbols=[kg_syms])
mods["boot_g"] = guid_df

pred_file2 = '/dfs/scratch1/lorr1/projects/bootleg/logs_guid/ent_only_wl_socc_air/2021_01_30/16_48_10/b51ac111/test/checkpoint_2.0/bootleg_labels.jsonl'

guid_df_ent = score_predictions(orig_file=input_dir / 'test.jsonl',
                 pred_file=pred_file2,
                 title_map=q2title,
                 cands_map=a2q,
                 type_symbols=[types_hy, types_wd, types_rel],
                 kg_symbols=[kg_syms])
mods["ent_g"] = guid_df_ent

In [21]:
for k, df in mods.items():
    if "cands" in df:
        df["num_cands"] = df["cands"].apply(lambda x: len(x))
        df["cand_names"] = df["cands"].apply(lambda x: [y[0] for y in x])
        df["cand_probs"] = df["cands"].apply(lambda x: [y[1] for y in x])
        del df["cands"]
    df["sentence_split"] = df["sentence"].apply(lambda x: x.split())
    df["span"] = df["span"].apply(lambda x: tuple(x))
    df["in_cand"] = df.apply(lambda x: x["gold_title"] in x["cand_names"], axis=1)
    df["qid_cnt"] = df["gold_qid"].apply(lambda x: qid2cnt.get(x, 0))
    df["pred_qid_cnt"] = df["pred_qid"].apply(lambda x: qid2cnt.get(x, 0))
    df["pred_qid_cnt"] = df["pred_qid"].apply(lambda x: qid2cnt.get(x, 0))
    mods[k] = df

In [33]:
# Testing out LFs

def is_team_pd(row):
    title = row["gold_title"]
    t = title.lower()
    in_list = ["national", "team"]
    not_in_list = ["competition", "season", "cup", "national team nomenclature", "teamsters"]
    r = all(i in t for i in in_list) and all(i not in t for i in not_in_list)
    if r:
        return r
    title = row["pred_title"]
    t = title.lower()
    in_list = ["national", "team"]
    not_in_list = ["competition", "season", "cup", "national team nomenclature", "teamsters"]
    r = (all(i in t for i in in_list) and all(i not in t for i in not_in_list))
    return r

def is_soccer_team(row):
    t = row["gold_title"].lower()
    in_list = ["national", "football", "team"]
    not_in_list = [
        "competition",
        "season",
        "cup",
        "national team nomenclature",
        "teamsters",
    ]
    r = all(i in t for i in in_list) and all(i not in t for i in not_in_list)
    return r

def around_punc(row):
    sent_split = row["sentence_split"]
    span_l, span_r = row["span"]
    punc = {"``", "''"}
    if len(punc.intersection(sent_split[span_l-2:span_l])) > 0 or len(punc.intersection(sent_split[span_r:span_r+2])) > 0:
        return True
    return False

def university_ty(row, key="wikidata_types_1229_gld"):
    gld_types = row[key]
    typs = {"university", "educational institution", "college", "school"}
    bad_typs = {"college athletic conference"}
    for t in gld_types:
        if any(tp in t for tp in typs) and t not in bad_typs:
            return True
    return False

def airport_ty(row, key="wikidata_types_1229_gld"):
    gld_types = row[key]
    typs = {"airport"}
    bad_typs = {}
    for t in gld_types:
        if any(tp in t for tp in typs) and t not in bad_typs:
            return True
    return False

def location_ty(row, key="wikidata_types_1229_gld"):
    gld_types = row[key]
    typs = {"city", "country"}
    bad_typs = {}
    for t in gld_types:
        if any(tp in t for tp in typs) and t not in bad_typs:
            return True
    return False

def album_title(row):
    return "album" in row["gold_title"].lower() or "song" in row["gold_title"].lower()

def title_pren_in_sent(row):
    if "(" in row["gold_title"]:
        in_paren = row["gold_title"].split("(")[1].split(")")[0]
        if in_paren.lower() in row["sentence"].lower().split():
            return True
    return False

def title_in_sent(row):
    span_l, span_r = row["span"]
    if len(row["gold_title"].split()) > 1:
        if row["gold_title"].lower() == " ".join(row["sentence"].lower().split()[span_l:span_r]):
            return True
    return False

univ_keywords = {"studied at", "studied at the", "studies at", "studies at the", "educated at", "educated at the",
                 "graduated from", "graduated from the", "department at", "department at the", "degree from", "degree from the",
                 "attended", "attended the", "professor at", "professor at the", "taught at", "taught at the", "univeristy of"}
def univ_keys(row):
    typs = row["wikidata_types_1229_gld"]
    span_l, span_r = row["span"]
    sent_left = row["sentence_split"][span_l-3:span_l]
    for univ_k in univ_keywords:
        if univ_k == " ".join(sent_left[-len(univ_k.split()):]):
            return True
    return False
            
        
def temp_pred(row):
    return is_soccer_team(row)

In [36]:
# What I print to test a LF
temp = mods["ent"]
temp["pred"] = temp.apply(lambda x: temp_pred(x), axis=1)
to_dis = temp[(temp["pred_qid"] != temp["gold_qid"]) & (temp["pred"])]
error_sh = temp[(temp["pred_qid"] != temp["gold_qid"])].shape[0]
dis_sh = to_dis.shape[0]
pred_sh = temp[temp["pred"]].shape[0]
print(temp.shape[0], error_sh, dis_sh, dis_sh/error_sh, dis_sh/pred_sh)
display(to_dis[columns])

285931 56784 253 0.004455480417018878 0.163860103626943


,sentence,sent_idx,is_gold_label,alias,span,gold_title,gold_qid,pred_title,in_cand,pred_qid,qid_cnt,pred_qid_cnt,aliases,wikidata_types_1229_gld,wikidata_types_1229_pred,cand_names,cand_probs
840,"In their third and final game , Russia defeated Greece 2-1 .",758228,True,greece,"(9, 10)",Greece national football team,Q134925,Greece men's national basketball team,True,Q331640,864,296,[greece],[national association football team],[national sports team],"[Greece, Greek mythology, Ancient Greece, Super League Greece, Greek Basket League, Greek War of Independence, Greece national football team, Football League Greece, Macedonia (Greece), Greek Civil War, Kingdom of Greece, Battle of Greece, Hellenic Navy, Greco-Italian War, Greek junta, Classical Greece, Axis occupation of Greece, Greece men's national basketball team, Church of Greece, Greek government-debt crisis, Hellenic Air Force, Greek cuisine, Ottoman Greece, Greece national under-21 f...","[0.0321845636, 0.0001899839, 3.82287e-05, 0.0226366799, 0.0001285177, 0.0001108148, 0.2419474572, 0.000299731, 6.55627e-05, 6.9603e-06, 2.85573e-05, 3.68239e-05, 0.000110786, 4.26064e-05, 1.95967e-05, 2.67994e-05, 1.59901e-05, 0.5141977072, 1.4953e-05, 0.0001248154, 0.0009965369, 4.45936e-05, 2.06049e-05, 0.1864584237, 2.69132e-05, 3.04932e-05, 9.57646e-05, 2.37304e-05, 1.44238e-05, 6.14033e-05]"
8975,Australia met Brazil in their second Group F game in Munich on 18 June .,781468,True,brazil,"(2, 3)",Brazil national football team,Q83459,Brazil women's national football team,True,Q757014,85,28,"[brazil, munich]",[national association football team],[women's national association football team],"[Brazil, Brazil national football team, Campeonato Brasileiro Série A, Brazilians, Brazil women's national football team, Empire of Brazil, Brazilian Navy, Brazilian Football Confederation, Military dictatorship in Brazil, Colonial Brazil, Football in Brazil, Brazil national under-20 football team, Brazil men's national basketball team, Constitution of Brazil, Music of Brazil, Brazil (1985 film), Brazil national under-23 football team, Brazil Squadron, Dutch Brazil, Brazil women's national v...","[0.073721014, 0.3629779518, 0.0003633957, 0.0001824134, 0.4986165762, 0.0002468404, 0.0004006325, 0.000205679, 0.0002003806, 0.0002112051, 0.0001742689, 0.0072335959, 0.0528873466, 8.61271e-05, 0.0002160669, 0.000265742, 9.70845e-05, 0.0001847276, 0.0001786653, 9.41582e-05, 0.0002847928, 0.0001456073, 0.0001236833, 0.0001606768, 8.91585e-05, 0.0001358843, 9.21316e-05, 0.000160872, 0.000156909, 0.0001064859]"
9315,"Nikos Tsiantakis 's appearance at the 1994 World Cup rounded off Nikos Tsiantakis 's national team career , which lasted from 1988 to 1994 , giving Nikos Tsiantakis 47 caps and 2 international goals .",782280,True,national team,"(14, 16)",Greece national football team,Q134925,Ireland national rugby union team,False,Q599903,864,261,"[tsiantakis, 1994 world cup, tsiantakis, national team, tsiantakis]",[national association football team],[national rugby union team],"[England national football team, Scotland national football team, New Zealand national rugby union team, Germany national football team, Italy national football team, Brazil national football team, United States men's national soccer team, Australia national cricket team, France national football team, Spain national football team, Argentina national football team, India national cricket team, Netherlands national football team, Wales national rugby union team, Republic of Ireland national f...","[0.0083726207, 0.0295856763, 0.0404866375, 0.0131809795, 0.007699484, 0.0250097644, 0.0552609377, 0.0012091803, 0.0258169509, 0.050889682, 0.018396182, 0.0560190268, 0.0164397955, 0.0379931815, 0.0111297704, 0.0826239362, 0.0158744939, 0.0154920388, 0.0319024324, 0.0364510231, 0.044805266, 0.0823890641, 0.0454786085, 0.0415028743, 0.0186212156, 0.0411132686, 0.0138756689, 0.018677827, 0.0130335297, 0.100668788]"
14428,"During the initial stages o

In [37]:
# What I print to test a LF
temp = mods["ent_g"]
temp["pred"] = temp.apply(lambda x: temp_pred(x), axis=1)
to_dis = temp[(temp["pred_qid"] != temp["gold_qid"]) & (temp["pred"])]
error_sh = temp[(temp["pred_qid"] != temp["gold_qid"])].shape[0]
dis_sh = to_dis.shape[0]
pred_sh = temp[temp["pred"]].shape[0]
print(temp.shape[0], error_sh, dis_sh, dis_sh/error_sh, dis_sh/pred_sh)
display(to_dis[columns])

285931 54871 322 0.005868309307284358 0.20854922279792745


,sentence,sent_idx,is_gold_label,alias,span,gold_title,gold_qid,pred_title,in_cand,pred_qid,qid_cnt,pred_qid_cnt,aliases,wikidata_types_1229_gld,wikidata_types_1229_pred,cand_names,cand_probs
840,"In their third and final game , Russia defeated Greece 2-1 .",758228,True,greece,"(9, 10)",Greece national football team,Q134925,Greece men's national basketball team,True,Q331640,864,296,[greece],[national association football team],[national sports team],"[Greece, Greek mythology, Ancient Greece, Super League Greece, Greek Basket League, Greek War of Independence, Greece national football team, Football League Greece, Macedonia (Greece), Greek Civil War, Kingdom of Greece, Battle of Greece, Hellenic Navy, Greco-Italian War, Greek junta, Classical Greece, Axis occupation of Greece, Greece men's national basketball team, Church of Greece, Greek government-debt crisis, Hellenic Air Force, Greek cuisine, Ottoman Greece, Greece national under-21 f...","[0.0657375529, 1.19382e-05, 5.51036e-05, 0.0014547963, 3.35592e-05, 8.5178e-06, 0.1008318439, 0.0001978761, 1.8055e-06, 3.987e-07, 3.39394e-05, 2.46667e-05, 8.457e-07, 1.2992e-06, 1.1934e-06, 3.30608e-05, 8.04e-07, 0.8013042212, 5.886e-07, 0.0013011417, 0.0009726551, 0.0005507836, 2.2396e-06, 0.0271249656, 1.04137e-05, 1.2586e-05, 1.70121e-05, 9.7102e-06, 1.7038e-06, 0.0002628242]"
9315,"Nikos Tsiantakis 's appearance at the 1994 World Cup rounded off Nikos Tsiantakis 's national team career , which lasted from 1988 to 1994 , giving Nikos Tsiantakis 47 caps and 2 international goals .",782280,True,national team,"(14, 16)",Greece national football team,Q134925,Portugal national football team,False,Q267245,864,95,"[tsiantakis, 1994 world cup, tsiantakis, national team, tsiantakis]",[national association football team],[national association football team],"[England national football team, Scotland national football team, New Zealand national rugby union team, Germany national football team, Italy national football team, Brazil national football team, United States men's national soccer team, Australia national cricket team, France national football team, Spain national football team, Argentina national football team, India national cricket team, Netherlands national football team, Wales national rugby union team, Republic of Ireland national f...","[0.0017670112, 0.0277295783, 1.67758e-05, 0.0584468879, 0.0496840924, 0.0485377014, 0.0777179226, 6.84148e-05, 0.0577830411, 0.0688135624, 0.0427530743, 0.0011527675, 0.0452967659, 0.0002409038, 0.0250730496, 0.008482811, 0.0484714583, 0.0560531244, 0.0829693824, 0.0654109865, 0.0119680213, 0.0077812071, 0.0085020214, 4.09329e-05, 0.0166664515, 0.0521972515, 0.0048073404, 0.0704647824, 0.0580746271, 0.0030281374]"
10170,"Geoff made Geoff 's debut for Canada in a March 1991 North American Championship match against Mexico , but since this game was not regarded official Geoff won Geoff 's first senior cap coming on as a sub in an April 1992 friendly match against China .",784592,True,mexico,"(16, 17)",Mexico national football team,Q164089,Mexico women's national football team,True,Q599923,34,6,"[aunger, aunger, canada, mexico, aunger, aunger, china]",[national association football team],[women's national association football team],"[Mexico, Mexico City, New Spain, Mexico national football team, Liga MX, Lucha libre, Mexican League, State of Mexico, Mexican cuisine, Second French intervention in Mexico, Mexican Navy, Mexico women's national football team, Mexicana Universal, Mexican Football Federation, First Mexican Empire, Autódromo Hermanos Rodríguez, Second Mexican Empire, Cinema of Mexico, Mexico national baseball team, COVID-19 pandemic in Mexico, Tigres de Quintana Roo, Roman Catholic Archdiocese of Mexico, Mexic...","[0.0011361684, 1.68692e-05, 4.448e-07, 0.2481831014, 3.295e-06, 0.0006355531, 1.04e-07, 1.0072e-06, 3.20117e-05, 5.28e-08, 6.448e-07, 0.3738825023, 1.097e-07, 9.99e-08, 5.11e-08, 4.252e-07, 5.694e-07, 1.8408e-06, 5.15e-08, 6.48e-

# Junk

In [110]:
print("G:C, P:T", (254-201)/201)
print("G:T, P:C", (216-161)/216)

print("G:C, P:T", (190-154)/154)
print("G:T, P:C", (151-106)/151)

G:C, P:T 0.263681592039801
G:T, P:C 0.25462962962962965
G:C, P:T 0.23376623376623376
G:T, P:C 0.2980132450331126
